<a href="https://colab.research.google.com/github/BFvandy/EV_Buzz_Analysis/blob/main/EV_SW%26Reddit_Zeroshot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prep

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
cur_path = "/content/drive/My Drive/EV Buzz Analysis"
os.chdir(cur_path)
!pwd

/content/drive/My Drive/EV Buzz Analysis


# Load Transformer

In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.1 MB/s 
     |████████████████████████████████| 120 kB 64.8 MB/s 
     |████████████████████████████████| 6.6 MB 42.4 MB/s 


In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli", device=0)

# Load Data

In [ ]:
comments = pd.read_csv('EV_SW&Reddit_prefiltered_v2.csv', encoding_errors='ignore')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3,5,7,9,10,11,12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
pd.set_option('display.max_columns', None)
comments

,Make,Model,Source,Purchased,Purchased Year,Disposed,Disposed Model Year,Rejected,Rejected Model Year,Sentiment,Age Range,Gender,Ethnicity,Income Bracket,Text,dependable,lasts long,value for money,quality fit and finish,attractive styling,safe,retains resale value,driver comfort,fun to drive,advanced features,responsive handling,prestigious,dealerships,fuel efficient,quick acceleration,environmentally friendly,affordable,trusted,leader,responsible,confident,innovative,exciting,practical,adventurous,passionate,distinctive,youthful,aggressive
0,Nissan,Leaf,Strategic Window,Nissan Leaf S Plus,2022.0,(None),NaN,Chevrolet Bolt EV,2022.0,Sentiment Not Calculated,Age 47,Male,White,"$65,001 - $70,000",It's a great commuter car. It's comfortable an...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Volkswagen,ID,Strategic Window,Volkswagen ID.4 AWD,2021.0,BMW X3 xDrive 30e,2018.0,Hyundai Kona,2021.0,Sentiment Not Calculated,Age 58,Male,White,"$200,001 - $300,000",LOVE THE CAR BUT...\n1) Dealer was dishonest a...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Tesla,Model 3,Strategic Window,Tesla Model 3,2022.0,(None),NaN,Tesla Model 3,2019.0,Sentiment Not Calculated,Age 45,Male,White,"$65,001 - $70,000",The range and energy efficiency is excellent- ...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,Volkswagen,ID,Strategic Window,Volkswagen ID.4 AWD,2021.0,Toyota RAV4,2016.0,Volvo C40,2021.0,Sentiment Not Calculated,Age 74,Male,White,"$300,001 - $400,000",I preordered using the VW online site December...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Tesla,Model 3,Strategic Window,Tesla Model 3,2022.0,Nissan Maxima,1995.0,Ford Mustang Mach-E,2022.0,Sentiment Not Calculated,Age 72,Male,NaN,"$80,001 - $90,000",I just finished my trip in my 2022 Model 3 lon...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155313,Volkswagen,ID,Reddit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Not to jump to too many conclusions, but in ph...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
155314,Volkswagen,ID,Reddit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Mine went from 30-1 to 30-2, so must be gettin...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
155315,Volkswagen,ID,Reddit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I think they're administering the update at 30...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
155316,Volkswagen,ID,Reddit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yup. mine went 30-2 today also. i'm not far fr...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
comments.shape

(155318, 44)

In [ ]:
comments.dtypes

Make                         object
Model                        object
Source                       object
Purchased                    object
Purchased Year              float64
 Disposed                    object
Disposed Model Year         float64
Rejected                     object
Rejected Model Year         float64
Sentiment                    object
Age Range                    object
Gender                       object
Ethnicity                    object
Income Bracket               object
Text                         object
dependable                  float64
lasts long                  float64
value for money             float64
quality fit and finish      float64
attractive styling          float64
safe                        float64
retains resale value        float64
driver comfort              float64
fun to drive                float64
advanced features           float64
responsive handling         float64
prestigious                 float64
dealerships                 

In [ ]:
#all the labels
# 17 functional attributes
dependable_labels = ['dependable', 'not dependable']
lasts_labels = ['lasts long', 'not lasts long']
value_labels = ['value for money', 'not value for money']
quality_labels = ['quality fit and finish', 'no quality fit and finish']
attractive_labels = ['attractive styling', 'not attractive styling']
safe_labels = ['safe', 'not safe']
retains_labels	= ['retains resale value', 'not retains resale value']
driver_labels	= ['driver comfort', 'driver not comfortable']
fun_labels = ['fun to drive', 'not fun to drive']
advanced_labels = ['advanced features', 'no advanced features']
responsive_labels = ['responsive handling', 'not responsive handling']
prestigious_labels = ['prestigious', 'not prestigious']
dealerships_labels = ['good dealerships', 'bad dealerships']
fuel_labels = ['fuel efficient', 'not fuel efficient']
quick_labels = ['quick acceleration', 'slow acceleration']
environ_labels = ['environmentally friendly', 'not environmentally friendly']
affordable_labels = ['affordable', 'not affordable']

# 12 personality attributes
trust_label = ['trusted', 'not trusted']
leader_label = ['leader', 'not leader']
resp_label = ['responsible', 'not responsible']
conf_label = ['confident','not confident']
innov_label = ['innovative', 'not innovative'] 
excit_label = ['exciting', 'not exciting']
pract_label = ['practical', 'not practical']
adven_label = ['adventurous', 'not adventurous']
pass_label = ['passionate', 'not passionate']
dist_label = ['distinctive','not distinctive']
youth_label = ['youthful', 'not youthful']
aggre_label = ['aggressive', 'not aggressive']


In [ ]:
funct_labels = [dependable_labels, lasts_labels, value_labels, quality_labels, attractive_labels, safe_labels, retains_labels, driver_labels, fun_labels, advanced_labels, responsive_labels, prestigious_labels, dealerships_labels, fuel_labels, quick_labels, environ_labels, affordable_labels]
pers_labels = [trust_label, leader_label, resp_label, conf_label, innov_label, excit_label, pract_label, adven_label, pass_label, dist_label, youth_label, aggre_label]

In [ ]:
all_labels = funct_labels + pers_labels
all_labels

[['dependable', 'not dependable'],
 ['lasts long', 'not lasts long'],
 ['value for money', 'not value for money'],
 ['quality fit and finish', 'no quality fit and finish'],
 ['attractive styling', 'not attractive styling'],
 ['safe', 'not safe'],
 ['retains resale value', 'not retains resale value'],
 ['driver comfort', 'driver not comfortable'],
 ['fun to drive', 'not fun to drive'],
 ['advanced features', 'no advanced features'],
 ['responsive handling', 'not responsive handling'],
 ['prestigious', 'not prestigious'],
 ['good dealerships', 'bad dealerships'],
 ['fuel efficient', 'not fuel efficient'],
 ['quick acceleration', 'slow acceleration'],
 ['environmentally friendly', 'not environmentally friendly'],
 ['affordable', 'not affordable'],
 ['trusted', 'not trusted'],
 ['leader', 'not leader'],
 ['responsible', 'not responsible'],
 ['confident', 'not confident'],
 ['innovative', 'not innovative'],
 ['exciting', 'not exciting'],
 ['practical', 'not practical'],
 ['adventurous', 'no

In [ ]:
len(all_labels)

29

# Implement Zero-shot Model

In [ ]:
for i in range(len(comments)):
  for j in range(29):
    #29 labels in total
    column_num = j + 15
    #match column index with label index
    if comments.iat[i, column_num] != 0:
      try:
        res = classifier(comments.at[i, 'Text'], all_labels[j], multi_label=False)
        ind = res['labels'].index(all_labels[j][0])
        comments.iat[i, column_num] = res.get('scores')[ind]
      except:
        pass
  if i % 5000 == 0:
    print(i)
  else:
    pass


0


/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:1039: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000


In [ ]:
comments

,Make,Model,Source,Purchased,Purchased Year,Disposed,Disposed Model Year,Rejected,Rejected Model Year,Sentiment,Age Range,Gender,Ethnicity,Income Bracket,Text,dependable,lasts long,value for money,quality fit and finish,attractive styling,safe,retains resale value,driver comfort,fun to drive,advanced features,responsive handling,prestigious,dealerships,fuel efficient,quick acceleration,environmentally friendly,affordable,trusted,leader,responsible,confident,innovative,exciting,practical,adventurous,passionate,distinctive,youthful,aggressive
0,Nissan,Leaf,Strategic Window,Nissan Leaf S Plus,2022.0,(None),NaN,Chevrolet Bolt EV,2022.0,Sentiment Not Calculated,Age 47,Male,White,"$65,001 - $70,000",It's a great commuter car. It's comfortable an...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.678851,NaN,NaN,0.729782,NaN,NaN,NaN,0.546715,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Volkswagen,ID,Strategic Window,Volkswagen ID.4 AWD,2021.0,BMW X3 xDrive 30e,2018.0,Hyundai Kona,2021.0,Sentiment Not Calculated,Age 58,Male,White,"$200,001 - $300,000",LOVE THE CAR BUT...\n1) Dealer was dishonest a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.087211,NaN,NaN,0.335159,NaN,0.04211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Tesla,Model 3,Strategic Window,Tesla Model 3,2022.0,(None),NaN,Tesla Model 3,2019.0,Sentiment Not Calculated,Age 45,Male,White,"$65,001 - $70,000",The range and energy efficiency is excellent- ...,NaN,NaN,NaN,NaN,0.972416,NaN,NaN,NaN,NaN,0.971715,0.993411,NaN,NaN,0.996648,0.885976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.986941,NaN,NaN,NaN,NaN,NaN
3,Volkswagen,ID,Strategic Window,Volkswagen ID.4 AWD,2021.0,Toyota RAV4,2016.0,Volvo C40,2021.0,Sentiment Not Calculated,Age 74,Male,White,"$300,001 - $400,000",I preordered using the VW online site December...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.220282,NaN,0.667287,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Tesla,Model 3,Strategic Window,Tesla Model 3,2022.0,Nissan Maxima,1995.0,Ford Mustang Mach-E,2022.0,Sentiment Not Calculated,Age 72,Male,NaN,"$80,001 - $90,000",I just finished my trip in my 2022 Model 3 lon...,NaN,0.830974,NaN,NaN,0.266351,NaN,NaN,NaN,NaN,NaN,0.421355,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.300735,0.686943,0.666795,0.575978,0.152594,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155313,Volkswagen,ID,Reddit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Not to jump to too many conclusions, but in ph...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155314,Volkswagen,ID,Reddit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Mine went from 30-1 to 30-2, so must be gettin...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155315,Volkswagen,ID,Reddit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I think they're administering the update at 30...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155316,Volkswagen,ID,Reddit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yup. mine went 30-2 today also. i'm not far fr...,NaN,0.816481,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.71388,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
comments['Model'] = comments['Model'].replace({'Model3':'Model 3'})
comments['Model'] = comments['Model'].replace({'ModelY':'Model Y'})
comments['Model'] = comments['Model'].replace({'Ioniq5':'Ioniq 5'})
comments['Model'] = comments['Model'].replace({'KonaEV':'Kona EV'})

In [ ]:
comments.groupby('Model').mean()

,Purchased Year,Disposed Model Year,Rejected Model Year,dependable,lasts long,value for money,quality fit and finish,attractive styling,safe,retains resale value,driver comfort,fun to drive,advanced features,responsive handling,prestigious,dealerships,fuel efficient,quick acceleration,environmentally friendly,affordable,trusted,leader,responsible,confident,innovative,exciting,practical,adventurous,passionate,distinctive,youthful,aggressive
Model,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Bolt,2020.886592,2004.627015,1987.725434,0.811765,0.717807,0.892414,0.793344,0.914523,0.760244,0.742581,0.772738,0.962907,0.891472,0.867887,0.750323,0.637900,0.819239,0.823336,0.887028,0.870457,0.811481,0.652545,0.871259,0.733312,0.821859,0.886031,0.868256,0.716883,0.911767,0.804438,0.229927,0.523894
EV6,2022.000000,2015.293194,2021.776504,0.500935,0.712441,0.602077,0.659461,0.692814,0.523116,0.686379,0.492560,0.837742,0.787813,0.645047,0.578554,0.610577,0.392655,0.584851,0.434952,0.546133,0.664098,0.448649,0.800274,0.371357,0.519840,0.697929,0.714055,0.663533,0.725838,0.694876,0.235981,0.552574
F150 Lightening,NaN,NaN,NaN,0.388645,0.704368,0.514738,0.549513,0.562451,0.415132,0.659042,0.391200,0.751469,0.738481,0.567160,0.506736,0.550579,0.324054,0.560072,0.339863,0.448919,0.585853,0.482622,0.781955,0.302793,0.478618,0.629914,0.641918,0.621038,0.680719,0.638775,0.212231,0.591386
Hummer,2022.000000,2015.000000,2022.000000,0.093408,0.754065,0.430295,0.547895,0.871397,0.382672,0.574996,0.285033,0.879769,0.825570,0.680479,0.724018,0.464865,0.416385,0.622197,0.440901,0.307425,0.518568,0.489803,0.899096,0.302995,0.768855,0.889908,0.768672,0.730845,0.781075,0.770653,0.330911,0.663070
ID,2021.000000,2014.508432,2021.004378,0.489343,0.716634,0.599932,0.657177,0.667816,0.486999,0.679467,0.523419,0.848338,0.769349,0.623643,0.569599,0.612018,0.367622,0.583245,0.458210,0.556861,0.652556,0.483100,0.782355,0.362099,0.544660,0.658092,0.682920,0.655956,0.731589,0.674586,0.255507,0.540235
Ioniq 5,2022.000000,2014.718023,2021.832402,0.445123,0.719897,0.545031,0.611871,0.664021,0.440277,0.651584,0.450220,0.791893,0.770366,0.602929,0.569412,0.595366,0.362546,0.597855,0.388596,0.473572,0.640962,0.483268,0.780461,0.323792,0.503921,0.647794,0.676510,0.649870,0.687388,0.674470,0.235599,0.579719
Kona EV,2020.597035,2012.410183,2010.547714,0.556229,0.713958,0.662796,0.679500,0.678290,0.535434,0.714374,0.536311,0.855343,0.800571,0.669849,0.609431,0.608542,0.418720,0.582259,0.489135,0.641642,0.696871,0.472796,0.803081,0.380505,0.547332,0.714181,0.722209,0.671783,0.758863,0.691821,0.278572,0.540580
Leaf,2021.022404,2001.492759,2013.271605,0.569685,0.669082,0.683458,0.694144,0.727035,0.592924,0.657108,0.614255,0.896759,0.804811,0.699579,0.658716,0.626797,0.493386,0.586908,0.645076,0.660525,0.718244,0.514458,0.828046,0.442112,0.624112,0.760016,0.764367,0.655196,0.794365,0.727155,0.262627,0.504569
Model 3,2021.105711,2008.664056,2011.380326,0.549949,0.707277,0.614938,0.626302,0.697434,0.566157,0.660837,0.526571,0.875662,0.834289,0.658766,0.637903,0.649875,0.475485,0.683882,0.575459,0.598204,0.693454,0.569370,0.781568,0.434034,0.643398,0.763646,0.719076,0.676885,0.783629,0.710091,0.239839,0.607109


In [ ]:
comments.replace(0, np.nan, inplace=True)
# replace 0 with nan

In [ ]:
comments.to_csv('EV_SW&Reddit_score_v5.csv',index = False)

In [ ]:
comments.to_excel('EV_SW&Reddit_score_v5.xlsx',index = False)